In [0]:
def get_gold_table_names():
    gold_tables_df = spark.sql("""
        SELECT table_name 
        FROM tracking_table 
        WHERE layer = 'Gold'
    """)
    return [row.table_name for row in gold_tables_df.collect()]

In [0]:
def dq_checks_gold(input_df):
    failure_log = []
    
    try:
        # Check for expected range of values
        out_of_range = input_df.filter((col("metric") < min_value) | (col("metric") > max_value)).collect()
        for row in out_of_range:
            failure_log.append((row["business_key"], "Metric out of range"))
        
        # Check for consistency in aggregated metrics
        inconsistent_metrics = input_df.groupBy("dimension").agg({"metric": "sum"}).filter("sum(metric) != expected_value").collect()
        for row in inconsistent_metrics:
            failure_log.append((row["dimension"], "Inconsistent metric values"))
        
        # Log failures to the dq_failure_log table
        if failure_log:
            for record_id, reason in failure_log:
                spark.sql(f"""
                    INSERT INTO dq_failure_log (layer, record_id, failure_reason)
                    VALUES ('Gold', '{record_id}', '{reason}')
                """)
    
    except Exception as e:
        # Log exceptions to the dq_failure_log table
        spark.sql(f"""
            INSERT INTO dq_failure_log (layer, record_id, failure_reason)
            VALUES ('Gold', NULL, 'Exception: {str(e)}')
        """)
    
    return failure_log

In [0]:
def log_to_table(table_name, layer, log_message, error_flag=False):
    timestamp = spark.sql("SELECT current_timestamp()").collect()[0][0]
    spark.sql(f"""
        INSERT INTO logging_table (table_name, layer, log_message, log_timestamp, error_flag)
        VALUES ('{table_name}', '{layer}', '{log_message}', '{timestamp}', {error_flag})
    """)

In [0]:
try:
    # Get the list of gold table names
    gold_table_names = get_gold_table_names()  # Assuming this function exists
        
    for table_name in gold_table_names:
        # Load the DataFrame for the current gold table
        input_df = spark.table(table_name)

        # Execute the gold data quality checks
        dq_checks_gold(input_df)

except Exception as e:
    log_to_table("Gold DQ check Execution", "Gold", f"Error during gold table operations: {str(e)}", error_flag=True)
    print(f"Error during gold table operations: {str(e)}")